In [1]:
import os
from libv3.utils import *
import pandas as pd
import json

In [2]:
############ configuration ################
############################################

CODE, BEHAVIOUR, THREAD, VER = get_config('mamba_config')   ### config stored in libv3/exp_config.txt

base_dir = '../trace_data' ### can be replaced with 'csv', 'exe_plot', 'histogram'
log_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR}'

print(log_path)

#### file to display
trace_file = 0

print('file number:', trace_file)

CODE: mamba
BEHAVIOUR: faulty_data
THREAD: single
VER: 3
../trace_data/mamba/single_thread/version_3/faulty_data
file number: 0


In [3]:
######### get paths #######################
paths_log, paths_traces, varlist_path, paths_label = get_paths(log_path)

### remove.Ds_store from all lists
paths_log = [x for x in paths_log if '.DS_Store' not in x]
paths_traces = [x for x in paths_traces if '.DS_Store' not in x]
varlist_path = [x for x in varlist_path if '.DS_Store' not in x]
paths_label = [x for x in paths_label if '.DS_Store' not in x]

paths_log.sort()
paths_traces.sort()
varlist_path.sort()

print(paths_log)
print(paths_traces)
print(varlist_path)
print(paths_label)

[]
['../trace_data/mamba/single_thread/version_3/faulty_data/trace_trial1', '../trace_data/mamba/single_thread/version_3/faulty_data/trace_trial2', '../trace_data/mamba/single_thread/version_3/faulty_data/trace_trial3', '../trace_data/mamba/single_thread/version_3/faulty_data/trace_trial4']
['../trace_data/mamba/single_thread/version_3/faulty_data/varlist_trial1.json', '../trace_data/mamba/single_thread/version_3/faulty_data/varlist_trial2.json', '../trace_data/mamba/single_thread/version_3/faulty_data/varlist_trial3.json', '../trace_data/mamba/single_thread/version_3/faulty_data/varlist_trial4.json']
['../trace_data/mamba/single_thread/version_3/faulty_data/labels/trace_trial1_labels.json', '../trace_data/mamba/single_thread/version_3/faulty_data/labels/trace_trial2_labels.json', '../trace_data/mamba/single_thread/version_3/faulty_data/labels/trace_trial3_labels.json', '../trace_data/mamba/single_thread/version_3/faulty_data/labels/trace_trial4_labels.json']


In [4]:
paths_traces[1]    ## 14, 17

'../trace_data/mamba/single_thread/version_3/faulty_data/trace_trial2'

In [5]:
## checking if all keys are constant
# for i in range(len(varlist_path)):
varlist1 = read_json(varlist_path[0])
varlist2 = read_json(varlist_path[1])
varlist3 = read_json(varlist_path[2])
varlist4 = read_json(varlist_path[3])
# print(varlist.keys())
keys = list(varlist1.keys())
keys.sort()
print(keys)
for k in keys:
    print(varlist1[k], varlist2[k], varlist3[k], varlist4[k])
    # print(k)
    if not (varlist1[k] == varlist2[k] == varlist3[k] == varlist4[k]): 
        print(k)


['0-0-0-AM2301_1', '0-0-0-AM2301_1_ADRR', '0-0-0-AM2301_2', '0-0-0-AM2301_2_ADRR', '0-0-0-AM2301_3', '0-0-0-AM2301_3_ADRR', '0-0-0-AM2301_4', '0-0-0-AM2301_4_ADRR', '0-0-0-BMP', '0-0-0-CONNECTION_A1', '0-0-0-CONNECTION_A2', '0-0-0-CONNECTION_A3', '0-0-0-CONNECTION_A4', '0-0-0-CONNECTION_BMP', '0-0-0-CONNECTION_CO', '0-0-0-CONNECTION_CO2', '0-0-0-CONNECTION_O2', '0-0-0-CONNECTION_VAR', '0-0-0-CO_ADRR', '0-0-0-FAILED_LORA', '0-0-0-FUNC_VAR', '0-0-0-I2CBUS', '0-0-0-LIMITS_BROKEN', '0-0-0-MAX_QUEUE', '0-0-0-MCP_CO', '0-0-0-MCP_O2', '0-0-0-O2_ADRR', '0-0-0-SCD30_ADRR', '0-0-0-SENSORBOARD_ID', '0-0-0-SENSORS_LIST', '0-0-0-SENSOR_DATA', '0-0-0-SENSOR_STATUS', '0-0-0-SPI_BUS', '0-0-0-THRESHOLD_LIMITS', '0-0-0-_pkng_frmt', '0-0-0-am_hum', '0-0-0-am_temp', '0-0-0-cb_30_done', '0-0-0-cb_retrans_done', '0-0-0-current_time', '0-0-0-func_call', '0-0-0-hb_msg', '0-0-0-j', '0-0-0-lora', '0-0-0-msg', '0-0-0-msg_interval', '0-0-0-que', '0-0-0-reading_co2', '0-0-0-retransmit_count', '0-0-0-retx_interval'

In [ ]:
############# check varlist is consistent ############
############# only for version 3 ######################

if VER == 3:
    to_number = is_consistent(varlist_path)

    if to_number != False:
        from_number = mapint2var(to_number)


In [ ]:
to_number

In [ ]:
############ Get variable list ######################
sorted_keys = list(from_number.keys())
sorted_keys.sort()
var_list = [from_number[key] for key in sorted_keys]   ### get the variable list
# print(var_list)

In [ ]:
sorted_keys

## Trace Plots

### Process Traces

In [ ]:
########## process the traces ###########
col_data = preprocess_traces(paths_traces, var_list)   ### in the format (trace_name, x_data, y_data, y_labels, trace_path) 

### Generate plot trace data

In [ ]:
############### preprocessing data to plot using plotly ##############
'''
Restructure the data in dictionary with (keys,value) pair :-  (time, timestamps) , (trace_name, trace)
'''
all_df = get_dataframe(col_data)   

In [ ]:
all_df[2]

In [ ]:

### plot single trace
for i, df in enumerate(all_df):
    
    if i == trace_file:
        plot_single_trace(df, sorted_keys, with_time=False, is_xticks=False)

In [ ]:
### get timestamp

timestamp = index2timestamp(all_df[0], 227)
print(timestamp)

## Execution Interval Plots

### Preprocess data

In [ ]:
####### preprocessing data ########

#### extract timestamps for each variable and store them in a dictionary along with index values for each variable in event trace

var_timestamps = get_var_timestamps(paths_traces)    #### in format (filename, dict of timestamps and index values)

to_plot = preprocess_variable_plotting(var_timestamps, var_list, from_number, trace_number=trace_file)   ### restructure the data for plotting


In [ ]:
len(to_plot[6][-1])
to_plot[6][-1]

### Generate execution interval plots

In [ ]:
### plot the data
plot_execution_interval_single(to_plot, is_xticks=False)

## Plot Labels

In [ ]:
### count and prepare labels to plot
'''
labels are of format [index1, index2, timestamp1, timestamp2, class]
'''
class_count = defaultdict(int)
for i, path in enumerate(paths_label):
    label_content = prepare_gt(path)
    ind, ts, cls = label_content
    # print(ind, ts, cls)
    for c in cls:
        class_count[c]+=1
        
    if i == trace_file:
        print(path)
        toplot_gt = label_content

    print(os.path.split(path)[-1], class_count)

    # break
for key, val in class_count.items():
    print(key, val)

In [ ]:
### plot
for i, df in enumerate(all_df):
    if i == trace_file:
        plot_obj = plot_single_trace(df, var_list, with_time=False, is_xticks=True, ground_truths=toplot_gt)
        plot_obj.show()

## Generate train data

In [ ]:
'''
Add the indices for the interval in 'normal_seq_inter' that show correct behvaiour in the traces. 
The format is as follows:
path_traces = [path1, path2, ...]
normal_seq_inter = [ ( intervals for trace1, ...), 
                    ( intervals for trace2 ...),
                      ...] 
'''

# normal_seq_inter = (  ( (0,110), (150,900), (1250,2000), (2050,2300) ),  ### v3, normal trace0
#             )

# for p, n_inter in zip(paths_traces, normal_seq_inter):
#     trace = read_traces(p)
#     train_data_path = os.path.join(os.path.dirname(p), 'train_data')

#     if not os.path.exists(train_data_path):
#         os.makedirs(train_data_path)

#     for i, inter in enumerate(n_inter):
#         start, end = inter
#         # trace[start:end].to_csv(os.path.join(train_data_path, f'interval_{start}_{end}.csv'), index=False)
#         json.dump(trace[start:end], open(os.path.join(train_data_path, f'interval_{start}_{end}.json'), 'w'))

In [ ]:
normal_seq_inter[0]

## Generate train data

In [ ]:
'''
Add the indices for the interval in 'normal_seq_inter' that show correct behvaiour in the traces. 
The format is as follows:
path_traces = [path1, path2, ...]
normal_seq_inter = [ ( intervals for trace1, ...), 
                    ( intervals for trace2 ...),
                      ...] 
'''

normal_seq_inter = (  ( (0,110), (150,900), (1250,2000), (2050,2300) ),)  ### v3, mamba2 trial1
            

for p, n_inter in zip(paths_traces, normal_seq_inter):
    trace = read_traces(p)
    train_data_path = os.path.join(os.path.dirname(p), 'train_data')

    if not os.path.exists(train_data_path):
        os.makedirs(train_data_path)

    for i, inter in enumerate(n_inter):
        start, end = inter
        # trace[start:end].to_csv(os.path.join(train_data_path, f'interval_{start}_{end}.csv'), index=False)
        json.dump(trace[start:end], open(os.path.join(train_data_path, f'interval_{start}_{end}.json'), 'w'))